# Visualize Effects of Changing Forest Type Groups



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:

carbon_file = "dat/fulljoin_carbon_area.csv"
tdee_file = 'dat/RF_table_n10_with_cumTDEE.csv'

carbon = pd.read_csv(carbon_file)
carbon_attributes = ["ForestTypeGroup", "AgeGroup", "carbon",  "carbon_se"]
carbon = carbon[carbon_attributes]

tdee = pd.read_csv(tdee_file)
tdee_attributes = ["ForestTypeGroup", "albedoAgeGroup", "cumTDEE", "cumTDEE_ub", "cumTDEE_lb"]
tdee = tdee[tdee_attributes].rename(columns={"albedoAgeGroup": "AgeGroup"})

# merge the two datasets
carbon_tdee = carbon.merge(tdee, on=["ForestTypeGroup", "AgeGroup"], how="inner")
carbon_tdee['NetCarbon'] = carbon_tdee['carbon'] - carbon_tdee['cumTDEE']


gam_carbon_file = "dat/gam_fia_carbon.csv"
gam_tdee_file = 'dat/gam_albedo_carbon.csv'

gam_carbon = pd.read_csv(gam_carbon_file).rename(columns={"GamCarbon": "carbon"})
gam_tdee = pd.read_csv(gam_tdee_file).rename(columns={"GamCarbon": "cumTDEE"})

# merge the two datasets
gam_carbon_tdee = gam_carbon.merge(gam_tdee, on=["ForestTypeGroup", "AgeGroup"], how="inner")

gam_carbon_tdee['NetCarbon'] = gam_carbon_tdee['carbon'] - gam_carbon_tdee['cumTDEE']

fortypes = gam_carbon['ForestTypeGroup'].unique()

In [3]:
def plot_benefit_individual(df, gam_df, x_type, y_type):
  """visualize the benefit of two types

  Args:
      df (_type_): data frame of the dataset
      x_type (_type_): original forest type group
      y_type (_type_): target forest type group
  """
  
  df_xy = df[df['ForestTypeGroup'].isin([x_type, y_type])]
  df_xy = df_xy[df_xy['AgeGroup']<=100]
  gam_df_xy = gam_df[gam_df['ForestTypeGroup'].isin([x_type, y_type])]
  gam_df_xy = gam_df_xy[gam_df_xy['AgeGroup']<=100]
  
  # plt.figure(figsize=(12, 8))
  fig, axs = plt.subplots(2, 2, figsize=(15, 10))
  axs = axs.flatten()
  
  vnames = ('carbon', 'cumTDEE', 'NetCarbon')
  for i, v in enumerate(vnames):
    sns.lineplot(gam_df_xy, x="AgeGroup", y=v, ax=axs[i], hue="ForestTypeGroup", legend=False)
    sns.scatterplot(df_xy, x="AgeGroup", y=v, ax=axs[i], hue="ForestTypeGroup", legend=i==0)
  
  netx = df[df['ForestTypeGroup']==x_type][['AgeGroup', 'NetCarbon']]
  nety = df[df['ForestTypeGroup']==y_type][['AgeGroup', 'NetCarbon']]
  netxy = netx.merge(nety, on='AgeGroup', how="inner")
  netxy['deltaNetCarbon'] = -netxy['NetCarbon_x'] + netxy['NetCarbon_y']
  netxy = netxy[netxy['AgeGroup'] <= 100]
  
  gam_netx = gam_df[gam_df['ForestTypeGroup']==x_type][['AgeGroup', 'NetCarbon']]
  gam_nety = gam_df[gam_df['ForestTypeGroup']==y_type][['AgeGroup', 'NetCarbon']]
  gam_netxy = gam_netx.merge(gam_nety, on='AgeGroup', how="inner")
  gam_netxy['deltaNetCarbon'] = - gam_netxy['NetCarbon_x'] + gam_netxy['NetCarbon_y']
  gam_netxy = gam_netxy[gam_netxy['AgeGroup'] <= 100]
  
  sns.lineplot(gam_netxy, x="AgeGroup", y='deltaNetCarbon', ax=axs[3], color="black", legend=False)
  sns.scatterplot(netxy, x="AgeGroup", y='deltaNetCarbon', ax=axs[3], color="black", )
 
  fig.suptitle(f"{x_type} → {y_type}", fontsize=16, fontweight="bold")
  
  plt.show()

In [4]:
import ipywidgets as widgets
from IPython.display import display

In [5]:
dd_start_fortype = widgets.Dropdown(
  options=fortypes,
  value=None,
  description="Current:"
)

dd_new_fortype = widgets.Dropdown(
  options=fortypes,
  value=None,
  description="New:"
)

toolbar = widgets.VBox([dd_start_fortype, dd_new_fortype])
btn_run = widgets.Button(description="Update")
canvas = widgets.Output()

display(toolbar, canvas)

prev_start = '' # previous value for dd_start_fortype
prev_new = '' # previous value for dd_new_fortype

@canvas.capture(clear_output=True)
def update_figure(dd):
  if dd_start_fortype.value != None and dd_new_fortype.value != None:
    plot_benefit_individual(carbon_tdee, gam_carbon_tdee, 
                        dd_start_fortype.value, dd_new_fortype.value)


dd_start_fortype.observe(update_figure, names='value')
dd_new_fortype.observe(update_figure, names='value')


Output()